# **Gruppo "Deep Learning Warlords"**

Jean Paul Guglielmo **Baroni**\
Maurizio **Cerisola**\
Davide **Maran**

**INDEX:**
1. Utilities\
     1.1 Dataset creation\
     1.2 Dataset Dimension Analysis\
     1.3 GPU check\
     1.4 Misclassified images check

2. Image preprocessing

3. Ternary models\
     3.1 No transfer learning\
     3.2 InceptionResNetV2\
     3.3 DenseNet201 (K_3 Model)

4. Binary models\
     4.1 K_M model\
     4.2 K_F model
     
5. Bayesian ensemble



**1.1**\
NOTE: the MaskDataset folder should be placed in the same folder as this notebook.\
In order to load the database, we divided the training set in the 3 **subfolders "0", "1", "2"**.

**1.2**\
In order to set the CNN input size, it’s important to study the dataset images sizes since changing images proportions provides a loss of information.\
At first we used squared inputs, but after looking at the dataset composition, we switched to 612x408 and it proved to be a more effective choice.

**1.3**\
In our instance, we opted for **Colab GPUs**: we implemented a check to understand whether the accellerators were properly set and to possibly set the Memory Growth.

**1.4**\
The "getMisclassified" function allows us to visually check which images were misclassified by **showing them and building a confusion matrix**.

**2**\
To load the data and make them suitable for feeding the neural network, we use the ImageDataGenerator. We also agumented the data using a small shift, rotation and zoom range because we think that these transformation preserve the faces. Moreover, we used the horizontal flip since faces are nearly symmetric under this transformation, while they aren't under vertical flip.

**3.1**\
The first model we developed is a standard CNN composed of a conv2D-ReLU-MaxPool2D layer sequences, but the huge number of trainable parameters (TP) leads to overfitting and unsatisfactory results.
After many trials, we decided not to further investigate architectures without transfer learning.

**3.2**\
The first backbone which provided noteworthy results has been InceptionResNetV2 with weights taken from imagenet.\
At first we used a simple NN without fine tuning: results are slightly better than the ones of 3.1, but a lot can be done in order to improve.\
The first HP we started playing with, is the number of InceptionResNetV2 freezed layers, this is particularly important, since it will influence the number of TP and so it will higly affect overfitting problems.\
Another important choice is the architecture of the NN top. A very shallow top does not allow the CNN to achieve good classification results while a very deep top have overfitting problems even though droput layers might help to mitigate them. As regards the activation functions, we tried many of them and sigmoid proved to be the best performing one.

**3.3**\
DenseNet201 proved to be a better performing backbone than InceptionResNetV2.\
Just like in the previous case, we spent a lot of time with trial-and-error HP tuning, we also got many tips from the DenseNet201 reference paper (https://arxiv.org/abs/1608.06993). <br>
This was the best ternary model and we called it K_3.

**4**\
To improve our predictions, we created two models that have 2, instead of 3 outputs:

*   K_M detects if there is a mask in the picture, so it's trained only on classes "0" vs "2"
*   K_F detects unmasked faces, so it's trained only on classes "1" vs "2"

An intense trial and error procedure led to the used HP.

**4.1**\
The K_M model is based on DenseNet201: for binary classifiers we opted for few dense units and epochs, to avoid overfitting.

**4.2**\
The K_F model is instead based on InceptionResnetV2.

**5**\
The predictions of K_3, K_M, K_F were merged. Based on the interpretation of the outputs as posterior probabilities, we apply a Bayes enseble and choose the class c which maximizes the quantinity
	$$P(c|K_3)*P(c|K_M)*P(c|K_F)$$
In order to compute P('0'|K_M) we assume that if an image is said to contain a mask, then it is equally likely for it to be in class '1' or '2'. The same is done for the face detector, with the classes '0','2' instead. Thus, we take


*   P('0'|K_M='no mask')=1
*   P('1'|K_M='mask')=P('2'|K_MD='mask')=1/2
*   P('1'|K_F='no face')=1
*   P('0'|K_F='no face')=P('2'|K_FD='no face')=1/2

This is just an approximation but has given good results, improving our final test accuracy up to 95%.

# **0. Global Settings**


In [ ]:
# Loads Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 
# Sets working directory
#### IMPORTANT : EDIT THE FOLLOWING
os.chdir("/content/drive/MyDrive/AN2DL/Final")

Required Packages and Global Parameters:

In [ ]:
#!pip install progressbar
!pip install tensorflow

class_names = ["0","1","2"]
img_h, img_w= (408, 612) 
SEED = 123
data_dir = "MaskDataset/training/"
validation_split = 0.2

# **1. Utilities**

## **1.1 Creation of the dataset**

In [ ]:
# Rearranges everything into MaskDataset2
from shutil import move
import os
import json
import progressbar


# Uploads training json. class_map is the dict mapping images names to their class
f=open('MaskDataset/train_gt.json','r')
class_map=json.loads(f.read())

# Creates the subfolders
for cn in class_names:
    folder_path = "MaskDataset/training/"+cn
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

# Progress bar
bar = progressbar.ProgressBar(maxval=len(class_map), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
progress = 0

# Moves the files to the proper class folder
for key in class_map:
    start_path = "MaskDataset/training/" + key
    end_path = "MaskDataset/training/" + str(class_map[key]) + "/" + key
    if not os.path.exists(end_path) and os.path.exists(start_path): # no overwriting
        move(start_path, end_path)
    progress += 1
    bar.update(progress)
bar.finish()

print("Number of items in MaskDataset/training: "+str(len(os.listdir("MaskDataset/training"))))
print("Number of items in the subfolders:")
for cn in class_names:
    print("- MaskDataset2/training/"+cn+": "+str(len(os.listdir("MaskDataset/training/"+cn))))

[========================================================================] 100%


Number of items in MaskDataset/training: 3
Number of items in the subfolders:
- MaskDataset2/training/0: 1900
- MaskDataset2/training/1: 1897
- MaskDataset2/training/2: 1817


## **1.2 Images Dimension Analysis**

In [ ]:
# Computes training set images dimensions
import os
import pandas as pd
import progressbar
from PIL import Image

for cn in class_names:
  #Set directory 
  directory = "MaskDataset/training/"+cn

  #Number of files in the directory
  num_files = len([f for f in os.listdir(directory)if os.path.isfile(os.path.join(directory, f))])

  #Initialize dataframe
  ImgSizes_test = pd.DataFrame(columns=['#'])
  #initialize Progress bar
  bar = progressbar.ProgressBar(maxval=num_files, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
  bar.start()
  progress = 0
  #For each image
  for filename in os.listdir(directory):
      #Open it and take sizes
      im = Image.open(directory + "/" + filename)
      width, height = im.size
      #Update dataframe
      index = str(width) + "x" + str(height);
      if (index in ImgSizes_test.index):
        ImgSizes_test.loc[index] += 1;
      else:
        ImgSizes_test.loc[index] = [1];
      #Update progress bar
      progress += 1
      bar.update(progress)
  bar.finish()

  #Sort dataframe
  ImgSizes_test = ImgSizes_test.sort_values(by=['#'], ascending=False)
  #Display output 
  print("Class",cn)
  print(ImgSizes_test)

In [ ]:
# Computes test set images dimensions
import os
import pandas as pd
import progressbar
from PIL import Image

#Set directory 
directory = "MaskDataset/test"

#Number of files in the directory
num_files = len([f for f in os.listdir(directory)if os.path.isfile(os.path.join(directory, f))])

#Initialize dataframe
ImgSizes_test = pd.DataFrame(columns=['#'])
#initialize Progress bar
bar = progressbar.ProgressBar(maxval=num_files, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
progress = 0
#For each image
for filename in os.listdir(directory):
    #Open it and take sizes
    im = Image.open(directory + "/" + filename)
    width, height = im.size
    #Update dataframe
    index = str(width) + "x" + str(height);
    if (index in ImgSizes_test.index):
      ImgSizes_test.loc[index] += 1;
    else:
      ImgSizes_test.loc[index] = [1];
    #Update progress bar
    progress += 1
    bar.update(progress)
bar.finish()

#Sort dataframe
ImgSizes_test = ImgSizes_test.sort_values(by=['#'], ascending=False)
#Display output 
ImgSizes_test

## **1.3 Check on the GPUs**

In [ ]:
# GPU Check
import os
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
import datetime

# Set GPU memory growth 
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

1 Physical GPUs, 1 Logical GPUs


## **1.4 Checks for misclassified images**

In [ ]:
# Find wrongly classified images 
from PIL import Image
import os
import json
from tensorflow.keras.models import load_model
import progressbar
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from sklearn.metrics import confusion_matrix

#Recall:
# 0: no one with facemask
# 1: all with with facemask
# 2: someone with and someone without mask
class_names = ["0","1","2"]

def getMisclassified(model, graphics = True):
  # Unpacks the dict into 3 lists:
  image_names = [] # name of the file
  image_class = [] # name of the correct class (according to json)
  image_prediction = [] # name of the predicted class (according to the selected model)

  for cn in class_names:
    folder_path = "MaskDataset/training/"+cn
    imageslist = os.listdir(folder_path)
    bar = progressbar.ProgressBar(maxval=len(imageslist), widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    progress = 0
    for file in imageslist:
      image_names.append(file)
      image_class.append(cn)
      #Get model prediction
      img = Image.open(folder_path+"/"+file).convert('RGB')
      img=img.resize((img_w,img_h))
      img_array = np.array(img)
      img_array = np.expand_dims(img_array, 0)
      img_array = np.true_divide(img_array,255)
      v=model.predict(img_array)
      image_prediction.append(str(np.argmax(v)))
      progress += 1
      bar.update(progress)
    bar.finish()

  # Builds Confusion Matrix
  confusion = confusion_matrix(image_class, image_prediction, class_names)

  # Displays wrongly classified images
  if graphics:
    for inm, icl, ipr in zip(image_names, image_class, image_prediction):
      if icl != ipr:
        img = mpimg.imread("MaskDataset/training/"+icl+"/"+inm)
        imgplot = plt.imshow(img)
        plt.show()
        print(inm, "true class:", icl, "predicted:", ipr)
          
  return confusion


In [ ]:
lm = load_model("Models/IncRes_e15_sz612x408_fr777_easyTop_btc64_v3")

In [ ]:
getMisclassified(lm)

#**2. Image Preprocessing**


In [ ]:
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import Dense
from keras import Model
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:

def dataLoader(classes = ["0","1","2"], bs = 16, SEED=SEED):
  # Create training ImageDataGenerator object
  train_data_gen = ImageDataGenerator(rotation_range=10,
                                      width_shift_range=img_w//20,#10
                                      height_shift_range=img_h//20,#10
                                      zoom_range=0.1,
                                      horizontal_flip=True,
                                      # vertical_flip=True,   #meglio toglierlo secondo me 
                                      fill_mode='constant',
                                      cval=0,
                                      rescale=1./255,
                                      preprocessing_function=None,
                                      data_format='channels_last',
                                      validation_split=validation_split)
  
  # Create validation ImageDataGenerator object
  valid_data_gen = ImageDataGenerator(rescale=1./255,validation_split=validation_split) 
  
  
  # Create training Generator object
  training_dir=data_dir
  train_gen = train_data_gen.flow_from_directory(training_dir,
                                                batch_size=bs,
                                                classes=classes,
                                                class_mode='categorical',
                                                target_size=(img_h, img_w),
                                                shuffle=True, 
                                                subset='training',
                                                seed=SEED) 
  steps_per_epoch=len(train_gen)

  # Create validation Generator object
  valid_dir=data_dir
  valid_gen = valid_data_gen.flow_from_directory(valid_dir, #same as training directory
                                                 batch_size=bs,
                                                 classes=classes,
                                                 class_mode='categorical',
                                                 target_size=(img_h, img_w),
                                                 shuffle=True, #even in validation, it doesn't change much. It's better to let it see diverse data at each epoch 
                                                 subset='validation',
                                                 seed=SEED) # set as validation data
  validation_steps=len(valid_gen)


  # Create training Dataset object
  num_classes = len(classes) #both for taining and validation
  train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=([None, img_h, img_w, 3], [None, num_classes])).repeat()
  # Create validation Dataset object
  valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=([None, img_h, img_w, 3], [None, num_classes])).repeat()

  return train_dataset, valid_dataset, steps_per_epoch, validation_steps, valid_gen

# **3. Ternary Models**

## **3.1 No Transfer Learning**

In [ ]:
### Calls Loader
train_dataset, valid_dataset, steps_per_epoch, validation_steps, valid_gen = dataLoader(["0","1","2"], bs = 10)

Found 2974 images belonging to 2 classes.
Found 743 images belonging to 2 classes.


In [ ]:
# No Transfer Learning
saveName = "NoTransLearnMC_ep15_sz612x408_btc10"

## Architecture ##
start_f = 8 #Starting number of filters
depth = 4 #Depth of our CNN

tmodel = tf.keras.Sequential()

# Features extraction
for i in range(depth):
    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]
    tmodel.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    tmodel.add(tf.keras.layers.ReLU())
    tmodel.add(tf.keras.layers.MaxPool2D(pool_size=(3, 3)))
    start_f *= 2 #each time i double the number of filters 
    
# Classifier
tmodel.add(tf.keras.layers.Flatten()) #I vectorize the volume 
tmodel.add(tf.keras.layers.Dense(units=512, activation='relu')) #I apply a classifier
tmodel.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Optimization ##
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
tmodel.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
tmodel.build(input_shape=(None, img_h, img_w, 3))
tmodel.summary()

## Callbacks for ES and LR Plateau ##
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7,restore_best_weights=True)
LR_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

## Fitting ##
tmodel.fit(x=train_dataset,
           epochs=15,  
           steps_per_epoch=steps_per_epoch, 
           validation_data=valid_dataset,
           validation_steps=validation_steps,
           callbacks=[es_callback,LR_adapter_callback,]) 

## Save model ##
tmodel.save('Models/'+saveName)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 408, 612, 8)       224       
_________________________________________________________________
re_lu_5 (ReLU)               (None, 408, 612, 8)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 136, 204, 8)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 136, 204, 16)      1168      
_________________________________________________________________
re_lu_6 (ReLU)               (None, 136, 204, 16)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 45, 68, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 45, 68, 32)       

## **3.2 InceptionResNetV2**

In [ ]:
### Calls Loader
train_dataset, valid_dataset, steps_per_epoch, validation_steps, valid_gen = dataLoader(["0","1","2"], bs = 64)

Found 2974 images belonging to 2 classes.
Found 743 images belonging to 2 classes.


### **3.2.1 Transfer Learning**

In [ ]:
#@title
saveName = "IncRes_e15_sz612x408_noFineTuning_easyTop_btc64"

## Architecture ##
tmodel = tf.keras.Sequential()
#Bottom
arch =  tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(img_h, img_w, 3), pooling="avg") 
for layer in arch.layers:
  layer.trainable = False
tmodel.add(arch)

#Output
tmodel.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Optimization ##
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
tmodel.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
tmodel.build(input_shape=(None, img_h, img_w, 3))
tmodel.summary()

## Callbacks for ES and LR Plateau ##
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7,restore_best_weights=True)
LR_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

## Fitting ##
tmodel.fit(x=train_dataset,
           epochs=15,  
           steps_per_epoch=steps_per_epoch, 
           validation_data=valid_dataset,
           validation_steps=validation_steps,
           callbacks=[es_callback,LR_adapter_callback,]) 

## Save model ##
tmodel.save('/content/drive/My Drive/AN2DL/modelli/'+saveName)

219062272/219055592 [==============================] - 15s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
dense (Dense)                (None, 3)                 4611      
Total params: 54,341,347
Trainable params: 4,611
Non-trainable params: 54,336,736
_________________________________________________________________
Epoch 1/15
71/71 [==============================] - 3161s 45s/step - loss: 0.9281 - accuracy: 0.5806 - val_loss: 0.7868 - val_accuracy: 0.6640
Epoch 2/15
71/71 [==============================] - 285s 4s/step - loss: 0.7207 - accuracy: 0.6979 - val_loss: 0.6810 - val_accuracy: 0.7005
Epoch 3/15
71/71 [==============================] - 284s 4s/step - loss: 0.6473 - accuracy: 0.7315 - val_loss: 0.6264 - val_accuracy: 0.7148
Epoch 4/1

### **3.2.2 Transfer learning with fine tuning** 

In [ ]:
#@title
saveName = "IncRes_e15_sz612x408_fr777_easyTop_btc64_v3"

## Architecture ##
tmodel = tf.keras.Sequential()
#Bottom
arch =  tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(img_h, img_w, 3), pooling="avg") 
for layer in arch.layers[:777]:
  layer.trainable = False
tmodel.add(arch)
#Top
tmodel.add(tf.keras.layers.Dropout(0.2))
tmodel.add(Dense(units=128, activation='sigmoid'))
tmodel.add(Dense(units=32, activation='sigmoid'))

#Output
tmodel.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Optimization ##
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
tmodel.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
tmodel.build(input_shape=(None, img_h, img_w, 3))
tmodel.summary()

## Callbacks for ES and LR Plateau ##
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7,restore_best_weights=True)
LR_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

## Fitting ##
tmodel.fit(x=train_dataset,
           epochs=15,  
           steps_per_epoch=steps_per_epoch, 
           validation_data=valid_dataset,
           validation_steps=validation_steps,
           callbacks=[es_callback,LR_adapter_callback,]) 

## Save model ##
tmodel.save('Models/'+saveName)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
dropout_5 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               196736    
_________________________________________________________________
dense_16 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 99        
Total params: 54,537,699
Trainable params: 3,397,379
Non-trainable params: 51,140,320
_________________________________________________________________
Epoch 1/15
281/281 [==============================] - 319s 1s/step - loss: 0.6035 - accuracy: 0.7471

## **3.3 DenseNet 201 (K_3 Model)**

In [ ]:
### Calls Loader
train_dataset, valid_dataset, steps_per_epoch, validation_steps, valid_gen = dataLoader(["0","1","2"], bs = 64)

Found 2974 images belonging to 2 classes.
Found 743 images belonging to 2 classes.


In [ ]:
#Best performing trinomial model
saveName = "K_3"

## Architecture ##
tmodel = tf.keras.Sequential()
#Bottom
arch = tf.keras.applications.densenet.DenseNet201(include_top=False, weights='imagenet', input_shape=(img_h, img_w, 3), pooling="avg")
for layer in arch.layers[:650]:
  layer.trainable = False
tmodel.add(arch)
#Top
tmodel.add(tf.keras.layers.Dropout(0.5))
tmodel.add(Dense(units=128, activation='sigmoid'))
tmodel.add(Dense(units=32, activation='sigmoid'))

#Output
tmodel.add(tf.keras.layers.Dense(units=3, activation='softmax'))

## Optimization ##
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
tmodel.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
tmodel.build(input_shape=(None, img_h, img_w, 3))
tmodel.summary()

## Callbacks for ES and LR Plateau ##
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7,restore_best_weights=True)
LR_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

## Fitting ##
tmodel.fit(x=train_dataset,
           epochs=15,  
           steps_per_epoch=steps_per_epoch, 
           validation_data=valid_dataset,
           validation_steps=validation_steps,
           callbacks=[es_callback,LR_adapter_callback,]) 

## Save model ##
tmodel.save('Models/'+saveName)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 1920)              18321984  
_________________________________________________________________
dropout_2 (Dropout)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               245888    
_________________________________________________________________
dense_5 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 99        
Total params: 18,572,099
Trainable params: 2,394,627
Non-trainable params: 16,177,472
_________________________________________________________________
Epoch 1/15
281/281 [==============================] - 366s 1s/step - loss: 0.5998 - accuracy: 0.7718

# **4. Binary Models**

## **4.1 K_M Model**

In [ ]:
### Calls Loader
train_dataset, valid_dataset, steps_per_epoch, validation_steps, valid_gen = dataLoader(["0","2"], bs = 16)

Found 2974 images belonging to 2 classes.
Found 743 images belonging to 2 classes.


In [ ]:
#@title
saveName = "K_M"

## Architecture ##
tmodel = tf.keras.Sequential()
#Bottom
arch = tf.keras.applications.densenet.DenseNet201(include_top=False, weights='imagenet', input_shape=(img_h, img_w, 3))
for layer in arch.layers[:600]:
  layer.trainable = False
tmodel.add(arch)

#Top
tmodel.add(tf.keras.layers.GlobalAveragePooling2D(data_format=None))
tmodel.add(tf.keras.layers.Dropout(0.5))
tmodel.add(Dense(units=24,activation='sigmoid', activity_regularizer=tf.keras.regularizers.l2(5e-5)))
tmodel.add(tf.keras.layers.Dropout(0.5))
tmodel.add(Dense(units=6,activation='sigmoid', activity_regularizer=tf.keras.regularizers.l2(1e-4)))

#Output
tmodel.add(tf.keras.layers.Dense(units=2, activation='softmax'))

## Optimization ##
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
tmodel.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
tmodel.build(input_shape=(None, img_h, img_w, 3))
tmodel.summary()

## Callbacks for ES and LR Plateau ##
# For just 8 epochs the callbacks are not used
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7,restore_best_weights=True)
LR_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto', min_delta=0.01, cooldown=0, min_lr=0)

74842112/74836368 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 12, 19, 1920)      18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 24)                46104     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 150       
_________________________________________________________

In [ ]:
## Fitting ##
tmodel.fit(x=train_dataset,
           epochs=8,
           steps_per_epoch=steps_per_epoch, 
           validation_data=valid_dataset,
           validation_steps=validation_steps,
           callbacks=[es_callback,LR_adapter_callback,]) 

Epoch 1/8
186/186 [==============================] - 251s 1s/step - loss: 0.5238 - accuracy: 0.8924 - val_loss: 0.3936 - val_accuracy: 0.9529
Epoch 2/8
186/186 [==============================] - 240s 1s/step - loss: 0.3553 - accuracy: 0.9684 - val_loss: 0.2677 - val_accuracy: 0.9744
Epoch 3/8
186/186 [==============================] - 240s 1s/step - loss: 0.2473 - accuracy: 0.9768 - val_loss: 0.1741 - val_accuracy: 0.9879
Epoch 4/8
186/186 [==============================] - 241s 1s/step - loss: 0.1704 - accuracy: 0.9855 - val_loss: 0.1335 - val_accuracy: 0.9852
Epoch 5/8
186/186 [==============================] - 237s 1s/step - loss: 0.1360 - accuracy: 0.9852 - val_loss: 0.1135 - val_accuracy: 0.9812
Epoch 6/8
186/186 [==============================] - 236s 1s/step - loss: 0.1034 - accuracy: 0.9896 - val_loss: 0.1141 - val_accuracy: 0.9785
Epoch 7/8
186/186 [==============================] - 238s 1s/step - loss: 0.0872 - accuracy: 0.9879 - val_loss: 0.1064 - val_accuracy: 0.9798
Epoch 

In [ ]:
tmodel.save('Models/'+saveName)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Models/K_M/assets


## **4.2 K_F Model**

In [ ]:
### Calls Loader
train_dataset, valid_dataset, steps_per_epoch, validation_steps, valid_gen = dataLoader(["1","2"], bs = 16)

In [ ]:
#KF model
saveName = "K_F"

## Architecture ##
tmodel = tf.keras.Sequential()
#Bottom
arch = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(img_h, img_w, 3), pooling = "avg")
for layer in arch.layers[:650]:
  layer.trainable = False
tmodel.add(arch)

#Top
tmodel.add(tf.keras.layers.Dropout(0.5))
tmodel.add(Dense(units=32,activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(1e-2)))

tmodel.add(tf.keras.layers.Dropout(0.5))
tmodel.add(Dense(units=32,activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(1e-2)))

# By multiple trials we decided to keep 2 dense layers with 32 units, indeed with a smaller number of classes, we don't need as many units

#Output 
tmodel.add(tf.keras.layers.Dense(units=2, activation='softmax'))

## Optimization ##
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
tmodel.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
tmodel.build(input_shape=(None, img_h, img_w, 3))
tmodel.summary()

## Callbacks for ES and LR Plateau ##
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7,restore_best_weights=True)
LR_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto', min_delta=0.01, cooldown=0, min_lr=0)


In [ ]:
### Fitting ##
tmodel.fit(x=train_dataset,
           epochs=12,
           steps_per_epoch=steps_per_epoch, 
           validation_data=valid_dataset,
           validation_steps=validation_steps,
           callbacks=[es_callback,LR_adapter_callback,]) 

In [ ]:
tmodel.save('Models/'+saveName)

# **5. Bayesian Ensemble**

In [ ]:
from tensorflow.keras.models import load_model

# Let us load our best models, they have different features: different output classes and input shapes
model_1=load_model('Models/K_3') # complete classifier
model02_1=load_model('Models/K_M') # distinguish between class 0 and 2
model12_1=load_model('Models/K_F') # distinguish between class 1 and 2

In [ ]:
# auxiliary of the Bayesian classifier
def converter02(v):
  g=v[0]
  w=np.zeros(3)
  w[0]=g[0]
  w[1]=g[1]/2
  w[2]=g[1]/2
  return w

# auxiliary of the Bayesian classifier
def converter12(v):
  g=v[0]
  w=np.zeros(3)
  w[0]=g[1]/2
  w[1]=g[0]
  w[2]=g[1]/2
  return w

### this function takes as input three models, the first distinguishes between classes 0 and 2, the second between 1 and 2 and the third between all the three.
### in order to put toghether the predictions, we use this function which applies Bayes theorem
def bayesian_classifier(v02_1,v12_1,v_1): 
  prior=np.array([1,1,1])   ### if the model has an high accuracy, it is better to use a flat prior that relies on the given results

  a=converter02(v02_1)
  b=converter12(v12_1)
  c=v_1[0]
  
  return np.argmax(a*b*c*prior)


In [ ]:
from PIL import Image
import os
from datetime import datetime



image_filenames = next(os.walk('MaskDataset/test'))[2]

results = {}
i=0

img_h=612
img_w=408

for image_name in image_filenames:
  
   img = Image.open('MaskDataset/test/'+image_name).convert('RGB')

   img=img.resize((img_h,img_w))
   img_array = np.array(img)
   img_array = np.expand_dims(img_array, 0)
   img_array = np.true_divide(img_array,255)

   v02_1=model02_1.predict(img_array)    # that's all for the 0 vs 2 classifier
   v12_1=model12_1.predict(img_array)    # that's all for the 1 vs 2 classifier
   v_1=model_1.predict(img_array)        # that's all for the three class model

   results[image_name]=str(bayesian_classifier(v02_1,v12_1,v_1))  # the Bayesian classifier takes the three posterior probabilities
                                                                  # and computes the winner


### how to create the resulting csv file, think you know it better than us :)
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

create_csv(results,'')